In [1]:
import stockutils, basicutils, config
import pandas_datareader.data as web, re, sqlite3, datetime

In [2]:
class StockData:
    def __init__(self, update = False):
        self.company_table = stockutils.company_table(update = update)
        
    @basicutils.memo
    def lookup(self, name, search_info = ['symbol', 'name', 'se'], se = None):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        conn = sqlite3.connect(config.sql_path)
        c = conn.cursor()
        result_table = []
        result_table.extend(stockutils.sql_search(name, search_info, c, se=se))
        if not result_table:
            print('Switching to regular expressions for ' + name + ' lookup.')
            found = self.fuzzy_search(name)
            if found:
                for item in found:
                    result_table.extend(stockutils.sql_search(item, search_info, c, se=se))
            else:
                result_table.append(('N/A', name))
        conn.close()
        return result_table
    
    #soon to be moved to matchutils.py
    def fuzzy_search(self,name):
        found_names = []
        for name in [name, name.upper()]:
            query = r".*" + r".*".join(list(name)) + r".*"
            for row in self.company_table.keys():
                match = re.search(query,row)
                if match:
                     found_names.append(match.group())
        return found_names
    
            
#       def stock_price(self, ticker, date):
            
        
        

In [3]:
basicutils.import_cache('data/cached_searches.json')

In [4]:
w = StockData()

In [5]:
w.lookup('Apple')

Switching to regular expressions for Apple lookup.


[('APDN', 'Applied DNA Sciences Inc', 'nasdaq'),
 ('AMAT', 'Applied Materials, Inc.', 'nasdaq'),
 ('ARCI', 'Appliance Recycling Centers of America, Inc.', 'nasdaq'),
 ('AIT', 'Applied Industrial Technologies, Inc.', 'nyse'),
 ('AAPL', 'Apple Inc.', 'nasdaq'),
 ('AGTC', 'Applied Genetic Technologies Corporation', 'nasdaq'),
 ('APLE', 'Apple Hospitality REIT, Inc.', 'nyse'),
 ('AMCC', 'Applied Micro Circuits Corporation', 'nasdaq'),
 ('AAOI', 'Applied Optoelectronics, Inc.', 'nasdaq'),
 ('GCP', 'GCP Applied Technologies Inc.', 'nyse'),
 ('SAIC', 'SCIENCE APPLICATIONS INTERNATIONAL CORPORATION', 'nyse')]

In [6]:
w.lookup('Nvidia',se = 'nasdaq')

Switching to regular expressions for Nvidia lookup.


[('NVDA', 'NVIDIA Corporation', 'nasdaq')]

In [7]:
basicutils.bulk_search(w.lookup,['Intel Corporation', 'Nvidia', 'Apple', 'Microsoft', 'Becton Dickinson'], se = 'nasdaq')

Switching to regular expressions for Apple lookup.
Switching to regular expressions for Microsoft lookup.
Switching to regular expressions for Becton Dickinson lookup.


[('INTC', 'Intel Corporation', 'nasdaq'),
 ('NVDA', 'NVIDIA Corporation', 'nasdaq'),
 ('APDN', 'Applied DNA Sciences Inc', 'nasdaq'),
 ('AMAT', 'Applied Materials, Inc.', 'nasdaq'),
 ('ARCI', 'Appliance Recycling Centers of America, Inc.', 'nasdaq'),
 ('AAPL', 'Apple Inc.', 'nasdaq'),
 ('AGTC', 'Applied Genetic Technologies Corporation', 'nasdaq'),
 ('AMCC', 'Applied Micro Circuits Corporation', 'nasdaq'),
 ('AAOI', 'Applied Optoelectronics, Inc.', 'nasdaq'),
 ('MSFT', 'Microsoft Corporation', 'nasdaq')]

In [8]:
w.lookup("Dick's")

Switching to regular expressions for Dick's lookup.


[('DKS', "Dick's Sporting Goods Inc", 'nyse')]

In [9]:
w.lookup('Becton',se = 'nyse')

Switching to regular expressions for Becton lookup.


[('JBT', 'John Bean Technologies Corporation', 'nyse'),
 ('BGE^B', 'Baltimore Gas & Electric Company', 'nyse'),
 ('BPI', 'Bridgepoint Education, Inc.', 'nyse'),
 ('BR', 'Broadridge Financial Solutions, Inc.', 'nyse'),
 ('WAB', 'Westinghouse Air Brake Technologies Corporation', 'nyse'),
 ('BTZ', 'BlackRock Credit Allocation Income Trust', 'nyse'),
 ('BNED', 'Barnes & Noble Education, Inc', 'nyse'),
 ('BAC', 'Bank of America Corporation', 'nyse'),
 ('BDX', 'Becton, Dickinson and Company', 'nyse'),
 ('BIT', 'BlackRock Multi-Sector Income Trust', 'nyse'),
 ('INF', 'Brookfield Global Listed Infrastructure Income Fund', 'nyse'),
 ('BSX', 'Boston Scientific Corporation', 'nyse'),
 ('BHE', 'Benchmark Electronics, Inc.', 'nyse'),
 ('IBM', 'International Business Machines Corporation', 'nyse')]

In [10]:
basicutils.bulk_search(w.lookup,['Intel', 'BD'])

Switching to regular expressions for Intel lookup.
Switching to regular expressions for BD lookup.


[('ICPT', 'Intercept Pharmaceuticals, Inc.', 'nasdaq'),
 ('IDTI', 'Integrated Device Technology, Inc.', 'nasdaq'),
 ('JMEI', 'Jumei International Holding Limited', 'nyse'),
 ('CXRX', 'Concordia International Corp.', 'nasdaq'),
 ('MSLI', 'Merus Labs International Inc.', 'nasdaq'),
 ('MATW', 'Matthews International Corporation', 'nasdaq'),
 ('INSW', 'International Seaways, Inc.', 'nyse'),
 ('NAII', 'Natural Alternatives International, Inc.', 'nasdaq'),
 ('AIN', 'Albany International Corporation', 'nyse'),
 ('IIPR', 'Innovative Industrial Properties, Inc.', 'nyse'),
 ('HNP', 'Huaneng Power International, Inc.', 'nyse'),
 ('WILC', 'G. Willi-Food International,  Ltd.', 'nasdaq'),
 ('DOOR', 'Masonite International Corporation', 'nyse'),
 ('CCI', 'Crown Castle International Corporation', 'nyse'),
 ('TASR', 'TASER International, Inc.', 'nasdaq'),
 ('VIGI', 'Vanguard International Dividend Appreciation ETF', 'nasdaq'),
 ('MTD', 'Mettler-Toledo International, Inc.', 'nyse'),
 ('SYUT', 'Synutra I

In [11]:
w.lookup('Intel Corporation', search_info = ['ipoyear', 'name'])

[('n/a', 'Intel Corporation')]

In [12]:
basicutils.bulk_search(w.lookup,['Blackrock Core Bond Trust', 'BlackRock Credit Allocation Income Trust'], search_info = ['ipoyear', 'name'])

[('2001', 'Blackrock Core Bond Trust'),
 ('2006', 'BlackRock Credit Allocation Income Trust')]

In [13]:
w.lookup('Becton, Dickinson and Company')

[('BDX', 'Becton, Dickinson and Company', 'nyse')]

In [14]:
basicutils.json_dump('data/cached_searches.json')

In [15]:
basicutils.clear_cache('data/cached_searches.json')